In [1]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
import re
from tensorflow.python.keras import backend
from tensorflow.python.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

In [4]:
from google.colab import drive #Dataset mounted from google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('/content/drive/MyDrive/poddf.csv')

In [6]:
df

,index,Name,Rating_Volume,Rating,Genre,Description
0,0,Fresh Air,10188,4.46133,Arts,"Fresh Air from WHYY, the Peabody Award-winning..."
1,0,The Moth,10154,4.69982,Performing,"Since its launch in 1997, The Moth has present..."
2,0,99% Invisible,12303,4.8693,Design,"Design is everywhere in our lives, perhaps mos..."
3,0,iFanboy.com Comic Book Podcast,1335,4.79551,Visual,The iFanboy.com Comic Book Podcast is a weekly...
4,0,Myths and Legends,11128,4.88282,Literature,"Jason Weiser tells stories from myths, legends..."
...,...,...,...,...,...,...
13627,0,Puromac,166,4.87952,Tech News,Puromac es una conversación sobre todo el mund...
13628,0,AVexcel,51,4.84314,Tech News,AVexcel is your guide to the best in home thea...
13629,0,Take a Network Break And Get The Latest Networ...,64,4.95313,Tech News,"Stay current with IT news on vendor moves, new..."
13630,0,AVexcel,52,4.82692,Tech News,AVexcel is your guide to the best in home thea...


In [7]:
df.Genre.value_counts().head(10) #visualizing top 10 genres

Business News             249
Investing                 245
Comedy                    244
Tech News                 243
Places & Travel           242
Educational Technology    241
Video Games               241
History                   241
Design                    241
News & Politics           240
Name: Genre, dtype: int64

In [8]:
df.Rating.value_counts().head(5)

Not Found    1887
5.0           702
4.5           144
4.66667       121
4.0           117
Name: Rating, dtype: int64

In [9]:
temp1=df[df["Genre"] == "Literature"]
temp2=df[df["Genre"] == "Business News"]
temp3=df[df["Genre"] == "Comedy"]
temp4=df[df["Genre"] == "History"]
temp5=df[df["Genre"] == "Places & Travel"]

In [10]:
merged_df = pd.concat([temp1, temp2, temp3,temp4,temp5]) #merge only 5 genres for better accuracy

In [11]:
merged_df.Genre.value_counts()

Business News      249
Comedy             244
Places & Travel    242
History            241
Literature         240
Name: Genre, dtype: int64

In [12]:
df=merged_df

In [13]:
df

,index,Name,Rating_Volume,Rating,Genre,Description
4,0,Myths and Legends,11128,4.88282,Literature,"Jason Weiser tells stories from myths, legends..."
8,0,The New Yorker: Fiction,1039,4.48412,Literature,A monthly reading and conversation with the Ne...
14,0,The NoSleep Podcast,5116,4.77853,Literature,The NoSleep Podcast is a multi-award winning a...
18,0,The Book Review,692,4.13295,Literature,The world's top authors and critics join host ...
21,0,MuggleCast: the Harry Potter podcast,2263,4.6814,Literature,MuggleCast is the most-listened to Harry Potte...
...,...,...,...,...,...,...
11392,0,The Vegas Tourist Podcast,9,2.44444,Places & Travel,Mark Anthony - Your podcast guide to Las Vegas...
11393,0,Disney Hipster Podcast,243,4.61728,Places & Travel,A podcast critiquing the aesthetic choices of ...
11394,0,Drinky Fun Time,88,4.84091,Places & Travel,It’s about drinking. Featuring celebrity guest...
11395,0,Capture The Magic - Disney World Podcast | Dis...,91,4.81318,Places & Travel,The Capture The Magic Podcast is a fun and inf...


In [14]:
df=merged_df.reset_index(drop=True) #reset the starting index from 0

In [15]:
df

,index,Name,Rating_Volume,Rating,Genre,Description
0,0,Myths and Legends,11128,4.88282,Literature,"Jason Weiser tells stories from myths, legends..."
1,0,The New Yorker: Fiction,1039,4.48412,Literature,A monthly reading and conversation with the Ne...
2,0,The NoSleep Podcast,5116,4.77853,Literature,The NoSleep Podcast is a multi-award winning a...
3,0,The Book Review,692,4.13295,Literature,The world's top authors and critics join host ...
4,0,MuggleCast: the Harry Potter podcast,2263,4.6814,Literature,MuggleCast is the most-listened to Harry Potte...
...,...,...,...,...,...,...
1211,0,The Vegas Tourist Podcast,9,2.44444,Places & Travel,Mark Anthony - Your podcast guide to Las Vegas...
1212,0,Disney Hipster Podcast,243,4.61728,Places & Travel,A podcast critiquing the aesthetic choices of ...
1213,0,Drinky Fun Time,88,4.84091,Places & Travel,It’s about drinking. Featuring celebrity guest...
1214,0,Capture The Magic - Disney World Podcast | Dis...,91,4.81318,Places & Travel,The Capture The Magic Podcast is a fun and inf...


In [17]:
def print_plot(index):
    example = df[df.index == index][['Description', 'Genre']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Genre:', example[1])
print_plot(60) #print description and genre at this index

ALOUD is the Library Foundation of Los Angeles' award-winning literary series of live conversations, readings and performances at the historic Central Library and locations throughout Los Angeles.
Genre: Literature


In [18]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
df['Description']

0       Jason Weiser tells stories from myths, legends...
1       A monthly reading and conversation with the Ne...
2       The NoSleep Podcast is a multi-award winning a...
3       The world's top authors and critics join host ...
4       MuggleCast is the most-listened to Harry Potte...
                              ...                        
1211    Mark Anthony - Your podcast guide to Las Vegas...
1212    A podcast critiquing the aesthetic choices of ...
1213    It’s about drinking. Featuring celebrity guest...
1214    The Capture The Magic Podcast is a fun and inf...
1215    RV Miles is a podcast for anyone interested in...
Name: Description, Length: 1216, dtype: object

In [20]:
df = df.reset_index(drop=True)
replace_by_space = re.compile('[/(){}\[\]\|@,;]')
bad_symbols = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):

    text = text.lower() # lowercase text
    text = replace_by_space.sub(' ', text) # replace 'replace_by_space' symbols by space in text and substitute the matched string with space.
    text = bad_symbols.sub('', text) # remove symbols which are in 'bad_symbols' from text and substitute the matched string with nothing. 
    text = text.replace('x', '')
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwords from text
    return text
df['Description'] = df['Description'].apply(clean_text)
df['Description'] = df['Description'].str.replace('\d+', '')

In [21]:
df

,index,Name,Rating_Volume,Rating,Genre,Description
0,0,Myths and Legends,11128,4.88282,Literature,jason weiser tells stories myths legends folkl...
1,0,The New Yorker: Fiction,1039,4.48412,Literature,monthly reading conversation new yorker fictio...
2,0,The NoSleep Podcast,5116,4.77853,Literature,nosleep podcast multiaward winning anthology s...
3,0,The Book Review,692,4.13295,Literature,worlds top authors critics join host pamela pa...
4,0,MuggleCast: the Harry Potter podcast,2263,4.6814,Literature,mugglecast mostlistened harry potter podcast h...
...,...,...,...,...,...,...
1211,0,The Vegas Tourist Podcast,9,2.44444,Places & Travel,mark anthony podcast guide las vegas beyond he...
1212,0,Disney Hipster Podcast,243,4.61728,Places & Travel,podcast critiquing aesthetic choices disney co...
1213,0,Drinky Fun Time,88,4.84091,Places & Travel,drinking featuring celebrity guests hosts dan ...
1214,0,Capture The Magic - Disney World Podcast | Dis...,91,4.81318,Places & Travel,capture magic podcast fun informative show dis...


In [22]:
df['Description'].values

array(['jason weiser tells stories myths legends folklore shaped cultures throughout history like stories aladdin king arthur hercules stories think know surprising origins others stories might heard really stories sourced world folklore retold modern earsthese stories wizards knights vikings dragons princesses kings time world beyond map dangerous wonderful place',
       'monthly reading conversation new yorker fiction editor deborah treisman',
       'nosleep podcast multiaward winning anthology series original horror stories rich atmospheric music enhance frightening tales',
       ...,
       'drinking featuring celebrity guests hosts dan dunn emma patterson mention drinking',
       'capture magic podcast fun informative show discuss aspects planning disney vacation latest disney related news disney trip reports disney tips tricks rumors',
       'rv miles podcast anyone interested rv camping lifestyle news reviews destinations recipes much rolling road motorhome trailer fifth wh

In [23]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 5000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['Description'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 10530 unique tokens.


In [24]:
X = tokenizer.texts_to_sequences(df['Description'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (1216, 250)


In [25]:
X

array([[   0,    0,    0, ..., 1725,  598,  182],
       [   0,    0,    0, ...,  426, 4466, 4467],
       [   0,    0,    0, ..., 1727, 2897,  105],
       ...,
       [   0,    0,    0, ..., 2761, 3354, 1192],
       [   0,    0,    0, ...,   55,  707, 1721],
       [   0,    0,    0, ..., 4328, 1822,  335]], dtype=int32)

In [26]:
df

,index,Name,Rating_Volume,Rating,Genre,Description
0,0,Myths and Legends,11128,4.88282,Literature,jason weiser tells stories myths legends folkl...
1,0,The New Yorker: Fiction,1039,4.48412,Literature,monthly reading conversation new yorker fictio...
2,0,The NoSleep Podcast,5116,4.77853,Literature,nosleep podcast multiaward winning anthology s...
3,0,The Book Review,692,4.13295,Literature,worlds top authors critics join host pamela pa...
4,0,MuggleCast: the Harry Potter podcast,2263,4.6814,Literature,mugglecast mostlistened harry potter podcast h...
...,...,...,...,...,...,...
1211,0,The Vegas Tourist Podcast,9,2.44444,Places & Travel,mark anthony podcast guide las vegas beyond he...
1212,0,Disney Hipster Podcast,243,4.61728,Places & Travel,podcast critiquing aesthetic choices disney co...
1213,0,Drinky Fun Time,88,4.84091,Places & Travel,drinking featuring celebrity guests hosts dan ...
1214,0,Capture The Magic - Disney World Podcast | Dis...,91,4.81318,Places & Travel,capture magic podcast fun informative show dis...


In [27]:
Y = pd.get_dummies(df['Genre']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (1216, 5)


In [28]:
Y

array([[0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]], dtype=uint8)

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 7)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1094, 250) (1094, 5)
(122, 250) (122, 5)


In [30]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 10
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Epoch 1/10
16/16 [==============================] - 18s 835ms/step - loss: 1.6076 - accuracy: 0.2441 - val_loss: 1.5922 - val_accuracy: 0.3909
Epoch 2/10
16/16 [==============================] - 13s 797ms/step - loss: 1.5716 - accuracy: 0.4664 - val_loss: 1.4712 - val_accuracy: 0.3273
Epoch 3/10
16/16 [==============================] - 13s 796ms/step - loss: 1.3689 - accuracy: 0.5063 - val_loss: 1.1838 - val_accuracy: 0.5727
Epoch 4/10
16/16 [==============================] - 13s 806ms/step - loss: 1.0177 - accuracy: 0.7688 - val_loss: 1.1143 - val_accuracy: 0.7000
Epoch 5/10
16/16 [==============================] - 13s 803ms/step - loss: 0.7453 - accuracy: 0.8515 - val_loss: 0.9627 - val_accuracy: 0.7273
Epoch 6/10
16/16 [==============================] - 13s 798ms/step - loss: 0.4556 - accuracy: 0.9277 - val_loss: 0.8361 - val_accuracy: 0.7818
Epoch 7/10
16/16 [==============================] - 13s 801ms/step - loss: 0.3078 - accuracy: 0.9383 - val_loss: 0.7057 - val_accuracy: 0.8091

In [31]:
new_summary  = ['I feel really depressed today and want to listen to something funny']
seq = tokenizer.texts_to_sequences(new_summary )
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
labels = ['Literature', 'Business News', 'Comedy' ,'History', 'Places & Travel']
print(pred, labels[np.argmax(pred)])


[[0.14756075 0.30124414 0.14381887 0.29555333 0.11182298]] Business News


In [32]:
pred_val = labels[np.argmax(pred)]
print(pred_val)

Business News


In [33]:
recommend =df[df["Genre"] == pred_val].head(5)

In [34]:
recommend

,index,Name,Rating_Volume,Rating,Genre,Description
240,0,Marketplace with Kai Ryssdal,2438,4.71575,Business News,marketplace leading business news program nati...
241,0,WSJ What's News,358,3.97207,Business News,top stories timely insights mirrored popular w...
242,0,Bloomberg Surveillance,382,3.86911,Business News,tom keene jon ferro pimm fo economy markets su...
243,0,A Conversation with Scott Galloway,9,4.88889,Business News,scott galloway professor marketing nyu stern f...
244,0,Economia,16,4.625,Business News,comentrios de bruno blecher carlos alberto sar...


In [35]:
new_summary  = ['I feel really unproductive today. Wish I had done more studies']
seq = tokenizer.texts_to_sequences(new_summary )
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
labels = ['Literature', 'Business News', 'Comedy' ,'History', 'Places & Travel']
print(pred, labels[np.argmax(pred)])

[[0.24429484 0.13954693 0.20082748 0.33464354 0.08068723]] History


In [36]:
pred_val1 = labels[np.argmax(pred)]
print(pred_val1)

History


In [37]:
recommend1 =df[df["Genre"] == pred_val1].head(5)

In [38]:
recommend1

,index,Name,Rating_Volume,Rating,Genre,Description
733,0,Whistlestop: Presidential History and Trivia,996,4.87751,History,john dickerson slates political gabfest revisi...
734,0,Alpha Male Buddhist From Brooklyn,Not Found,Not Found,History,tv lying students knowledge eastern philosophy...
735,0,Stuff You Missed in History Class,10927,4.16931,History,join holly tracy bring greatest strangest stuf...
736,0,Dan Carlin's Hardcore History,25469,4.80686,History,hardcore history journalist broadcaster dan ca...
737,0,Lore,22183,4.77988,History,lore biweekly podcast upcoming tv show dark hi...
